# Fetch Google News articles from November 15, 2019 to December 17, 2023.

The data is extracted by looking for the following websites:

- El Pais


In [2]:
import requests
from bs4 import BeautifulSoup
import json

# Get pages from El Pais


In [ ]:
def verify_data(data):
	try:
		data = data.text
		return data
	except:
		return "None"

def get_pages(term, document_path):

	search_term = term
	search_term = search_term.replace(' ', '+')  
	num_pages = 1
	json_pages_info = {
		"pages": []
	}

	headers = {'User-Agent': 'Chrome/58.0.3029.110'}

	all_pages_links = []

	for page in range(0, num_pages):
		url = f'https://elpais.com/buscador/Estallido%20Social%20en%20Chile/{page}'
		response = requests.get(url, headers=headers)

		if response.status_code == 200:
			print("status 200")

			response.encoding = 'ISO-8859-1'
			content = response.content.decode('ISO-8859-1')
			soup = BeautifulSoup(content, 'html.parser')
			
			with open('output.html', 'w', encoding='utf-8') as f:
				f.write(soup.prettify())

			news = soup.find("div", class_="bu_b _g-o").find_all('article')

			for page_section in news:

				timestamp = (int)(page_section["data-timestamp"])
				if (timestamp < 1573786800 or timestamp > 1702782000):
					print("La noticia no corresponde a la fecha solicitada")
					continue

				title = verify_data(page_section.find("header").find("h2"))
				category = verify_data(page_section.find("header").find("a"))
				description = verify_data(page_section.find("p", class_="c_d"))
				date = verify_data(page_section.find("div", class_="c_f"))
				image_link = page_section.find("figure").find("a", href=True)["href"]
				author = verify_data(page_section.find("div", class_="c_a"))

				link_section = page_section.find("header").find("h2").find('a', href=True)
				link = link_section['href']

				if (all_pages_links.count(link) > 0):
					print("Sitio ya ingresado")
					continue
				else:
					all_pages_links.append(link)

				link_info = {
					"newscast" : "El País",
					"title": title,
					"description": description,
					"category": category,
					"date": date,
					"image_link": image_link,
					"author": author,
					"link": link,
				}
				json_pages_info["pages"].append(link_info)
		else:
			print(f"Error al acceder a la página {page + 1}: {response.status_code}")
			
	print(f"Se han encontrado: {len(json_pages_info['pages'])} páginas")

	with open(f"{document_path}", 'w', encoding='utf-8') as file:
		json.dump(json_pages_info, file, ensure_ascii=False, indent=4)

In [ ]:
get_pages('Estallido Social Chile', './archive/elpais_pages_aux.json')

status 200
La noticia no corresponde a la fecha solicitada
La noticia no corresponde a la fecha solicitada
La noticia no corresponde a la fecha solicitada
La noticia no corresponde a la fecha solicitada
La noticia no corresponde a la fecha solicitada
La noticia no corresponde a la fecha solicitada
La noticia no corresponde a la fecha solicitada
Se han encontrado: 7 páginas


In [5]:
with open("archive/elpais_pages.json", 'r', encoding='utf-8') as file:
    file = json.load(file)
    print(len(file["pages"]))

44
